In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from config import api_key
c = Census(api_key, year=2017)
#!pip install census
Medicare_file = "medicare_charges.csv"

In [42]:
census_data = c.acs5.get(("NAME", "B19013_001E", 
                          "B01003_001E", 
                          "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E",
                         "B25077_001E",
                         "B15003_022E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "B25077_001E": "Median Home Value",
                                      "B15003_022E": "Bachelor Count",
                                      "NAME": "Name", "zip code tabulation area": "Zip Code"})
Census_Cleaned = census_pd[["Population",
                            "Median Age",
                            "Per Capita Income",
                            "Zip Code"]]
Census_Cleaned = Census_Cleaned.dropna()
Census_Cleaned.head(10)

Population  Median Age  Per Capita Income Zip Code
0     17599.0        38.9             7041.0    00601
1     39209.0        40.9             8978.0    00602
2     50135.0        40.4            10897.0    00603
3      6304.0        42.8             5960.0    00606
4     27590.0        41.4             9266.0    00610
5     62566.0        41.4            10283.0    00612
6     10687.0        43.7             9002.0    00616
7     24508.0        38.7             8762.0    00617
8      7405.0        48.4            11935.0    00622
9     42321.0        41.3             9101.0    00623

In [3]:
print(len(Census_Cleaned["Zip Code"]))


33120


In [27]:
Medicare_Cost_df = pd.read_csv(Medicare_file)
Medicare_Cost_Cleaned = Medicare_Cost_df[["Provider Zip Code",
                                          "Provider State", 
                                          "DRG Definition", 
                                          "Average Total Payments"]]
Medicare_Cost_Cleaned = Medicare_Cost_Cleaned.rename(columns = {"Provider Zip Code":"Zip Code",
                                                               "Provider State":"State",
                                                               "DRG Definition":"Procedure",
                                                               "Average Total Payments": "Cost"})
Medicare_Cost_Cleaned.head()
#print(len(Medicare_Cost_df))

Zip Code State                                          Procedure  \
0     36301    AL  023 - CRANIOTOMY W MAJOR DEVICE IMPLANT OR ACU...   
1     36301    AL  025 - CRANIOTOMY & ENDOVASCULAR INTRACRANIAL P...   
2     36301    AL  027 - CRANIOTOMY & ENDOVASCULAR INTRACRANIAL P...   
3     36301    AL                037 - EXTRACRANIAL PROCEDURES W MCC   
4     36301    AL                 038 - EXTRACRANIAL PROCEDURES W CC   

           Cost  
0  25823.714290  
1  20688.343750  
2  11811.181820  
3  19134.000000  
4   8519.470588

In [57]:
Medicare_Cost_Cleaned = Medicare_Cost_Cleaned[Medicare_Cost_Cleaned.State == "CA"]
Medicare_Cost_817 = Medicare_Cost_Cleaned[Medicare_Cost_Cleaned.Procedure == "871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W MCC"]
Medicare_Cost_817 = Medicare_Cost_817.groupby("Zip Code").mean()
Medicare_Cost_817["Zip Code"] = Medicare_Cost_817.index
#Medicare_Cost_817.reset_index(drop = True , inplace = True)


Medicare_Cost_Total = Medicare_Cost_817.rename(columns = {"Cost":"Total Avg 817"})

#Medicare_Cost_Total["Total Avg 817"] = Medicare_Cost_Cleaned[Medicare_Cost_Cleaned.Procedure == "871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W MCC"].groupby("Zip Code").mean()["Cost"]
Medicare_Cost_Total["Total Avg 291"] = round(Medicare_Cost_Cleaned[Medicare_Cost_Cleaned.Procedure == "291 - HEART FAILURE & SHOCK W MCC"].groupby("Zip Code").mean()["Cost"], 0)
Medicare_Cost_Total["Total Avg 872"] = round(Medicare_Cost_Cleaned[Medicare_Cost_Cleaned.Procedure == "872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W/O MCC"].groupby("Zip Code").mean()["Cost"], 0)
Medicare_Cost_Total["Total Avg 470"] = round(Medicare_Cost_Cleaned[Medicare_Cost_Cleaned.Procedure == "470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W/O MCC"].groupby("Zip Code").mean()["Cost"], 0)
Medicare_Cost_Total["Total Avg 392"] = round(Medicare_Cost_Cleaned[Medicare_Cost_Cleaned.Procedure == "392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS W/O MCC"].groupby("Zip Code").mean()["Cost"], 0)

Medicare_Cost_Total = Medicare_Cost_Total.dropna()
Medicare_Cost_Total.reset_index(drop = True , inplace = True)
Medicare_Cost_Total



Total Avg 817  Zip Code  Total Avg 291  Total Avg 872  Total Avg 470  \
0     22185.804120     90015        19343.0        15902.0        24933.0   
1     16186.630820     90017        13153.0         9645.0        18058.0   
2     17438.370855     90027        13887.0         9169.0        19586.0   
3     15827.859870     90028        12988.0         9306.0        17425.0   
4     26462.555853     90033        23416.0        16866.0        21003.0   
..             ...       ...            ...            ...            ...   
180   13586.240580     95966        11342.0         8155.0        15967.0   
181   13511.659690     95969        11305.0         8972.0        15886.0   
182   14965.846885     96001        12310.0         8846.0        16913.0   
183   16008.925930     96080        13438.0         9235.0        18595.0   
184   20814.500000     96150        17879.0        12177.0        28324.0   

     Total Avg 392  
0          13643.0  
1           7357.0  
2           8709.0  
3           7162.0  
4          14192.0  
..             ...  
180         6020.0  
181         5872.0  
182         6521.0  
183         8787.0  
184         8762.0  

[185 rows x 6 columns]

In [14]:
top5 = Medicare_Cost_Cleaned.Procedure.value_counts()
top5.head()

871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W MCC                    277
291 - HEART FAILURE & SHOCK W MCC                                           260
872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W/O MCC                  258
470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W/O MCC    236
392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS W/O MCC                228
Name: Procedure, dtype: int64

In [58]:
Census_Cleaned["Zip Code"] = Census_Cleaned["Zip Code"].astype(int)
Medicare_Cost_Total["Zip Code"] = Medicare_Cost_Total["Zip Code"].astype(int)
Data = pd.merge(Medicare_Cost_Total, Census_Cleaned, how="inner", on="Zip Code")
Data = Data.drop([8])
Data["Total Avg 817"] = round(Data["Total Avg 817"], 0)
Data.head(30)

Total Avg 817  Zip Code  Total Avg 291  Total Avg 872  Total Avg 470  \
0         22186.0     90015        19343.0        15902.0        24933.0   
1         16187.0     90017        13153.0         9645.0        18058.0   
2         17438.0     90027        13887.0         9169.0        19586.0   
3         15828.0     90028        12988.0         9306.0        17425.0   
4         26463.0     90033        23416.0        16866.0        21003.0   
5         13468.0     90036        11017.0         8015.0        16050.0   
6         21730.0     90048        16924.0        10737.0        18910.0   
7         19860.0     90057        15790.0        10861.0        18605.0   
9         14898.0     90241        12069.0         8923.0        16775.0   
10        12567.0     90291        10451.0         7249.0        15134.0   
11        14487.0     90301        12241.0         8888.0        17108.0   
12        17599.0     90404        13135.0         9025.0        16782.0   
13        47028.0     90502        46395.0        34811.0        37921.0   
14        14032.0     90503        11368.0         8661.0        16268.0   
15        14773.0     90602        12370.0         8507.0        16343.0   
16        15187.0     90640        12882.0         9226.0        17562.0   
17        13984.0     90712        11900.0         8220.0        16199.0   
18        13604.0     90720        10486.0         7443.0        14109.0   
19        14990.0     90732        11706.0         8124.0        16388.0   
20        13424.0     90804        11180.0         8066.0        14491.0   
21        15728.0     90806        12013.0         8734.0        17786.0   
22        18111.0     90813        15328.0        11799.0        20636.0   
23        14273.0     91006        11832.0         8254.0        18104.0   
24        16639.0     91204        13698.0         9795.0        18548.0   
25        16200.0     91206        13009.0         9542.0        17674.0   
26        15634.0     91307        11261.0         8338.0        16344.0   
27        15231.0     91325        12634.0         9331.0        17121.0   
28        16208.0     91355        13004.0         7660.0        15426.0   
29        13916.0     91356        11284.0         8099.0        16803.0   
30        15363.0     91360        11559.0         7802.0        16786.0   

    Total Avg 392  Population  Median Age  Per Capita Income  
0         13643.0     20187.0        32.4            31423.0  
1          7357.0     26390.0        30.8            21874.0  
2          8709.0     45616.0        38.0            46196.0  
3          7162.0     31054.0        34.4            32587.0  
4         14192.0     48841.0        30.0            13332.0  
5          6068.0     38116.0        34.4            53842.0  
6          9323.0     22860.0        36.7            70123.0  
7          8452.0     49389.0        34.1            15492.0  
9          6749.0     44097.0        35.6            25576.0  
10         6308.0     27170.0        38.4            71411.0  
11         6779.0     37518.0        33.7            18999.0  
12         6708.0     22586.0        37.9            45565.0  
13        23895.0     18010.0        43.2            28733.0  
14         6281.0     45349.0        40.3            40259.0  
15         8547.0     26482.0        32.4            21818.0  
16         7263.0     63547.0        36.8            22313.0  
17         6432.0     31471.0        38.7            32656.0  
18         5691.0     22318.0        43.2            43531.0  
19         6168.0     22408.0        45.7            44696.0  
20         5924.0     40225.0        31.2            24806.0  
21         7244.0     42587.0        33.2            19808.0  
22         9437.0     60935.0        28.3            14084.0  
23         6076.0     33200.0        43.2            42897.0  
24         7687.0     18318.0        35.8            21493.0  
25         7466.0     34938.0        41.7            33560.0  
26       

In [7]:
####Off to the races yall :)